###  Install required packages

In [1]:
import rdflib
import networkx as nx
from node2vec import Node2Vec
from sklearn.metrics.pairwise import cosine_similarity

###  Function to convert an OWL graph to NetworkX graph

In [2]:
def rdf_to_nx_graph(rdf_graph):
    G = nx.Graph()
    for subject, predicate, obj in rdf_graph:
        G.add_node(subject)
        G.add_node(obj)
        G.add_edge(subject, obj, label=predicate)
    return G

###  Paths to ontological files

In [3]:
micro_ontology_path = "ontology1.owl"
linklings_ontology_path = "ontology2.owl"

### Create RDF graphs for ontologies

In [4]:
micro_ontology = rdflib.Graph()
linklings_ontology = rdflib.Graph()

### Load and analyze ontologies from OWL files (in RDF/XML format)

In [5]:
micro_ontology.parse(micro_ontology_path, format="xml")
linklings_ontology.parse(linklings_ontology_path, format="xml")

<Graph identifier=Neeb38b7605de4748bf13ba72f2b53574 (<class 'rdflib.graph.Graph'>)>

### Convert from ontologies to Networkx graphs

In [6]:
micro_nx_graph = rdf_to_nx_graph(micro_ontology)
linklings_nx_graph = rdf_to_nx_graph(linklings_ontology)

### Create Node2vec models for the two graphs

In [7]:
node2vec_model_micro = Node2Vec(micro_nx_graph, dimensions=64, walk_length=30, num_walks=200, workers=4)
node2vec_model_linklings = Node2Vec(linklings_nx_graph, dimensions=64, walk_length=30, num_walks=200, workers=4)

Computing transition probabilities:   0%|          | 0/136 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/205 [00:00<?, ?it/s]

### Generate the embeddings for the nodes of the two graphs

In [8]:
embeddings_micro = node2vec_model_micro.fit(window=10, min_count=1, batch_words=4)
embeddings_linklings = node2vec_model_linklings.fit(window=10, min_count=1, batch_words=4)

### Measure the similarity between micro and linklings vectors (example of cosine similarity)

In [9]:
similarities = cosine_similarity(embeddings_micro.wv.vectors, embeddings_linklings.wv.vectors)

### Show the similarity matrix divide into two classes. Complex match (n, m)

In [10]:
print("\nSimilarity matrix between micro and linklings vectors:")
print(similarities)


Similarity matrix between micro and linklings vectors:
[[-0.1399865   0.02260146  0.07209042 ... -0.04796334  0.05942156
   0.2130788 ]
 [-0.01146752  0.15662952 -0.103155   ...  0.16304547 -0.12799382
  -0.05843265]
 [-0.0160518   0.1366581  -0.22955324 ... -0.16813566 -0.13405496
  -0.18371838]
 ...
 [-0.06704433 -0.10639544  0.09555212 ... -0.01297597  0.07766649
   0.16425095]
 [ 0.18679337  0.2548368   0.05702506 ... -0.06764852 -0.00329821
  -0.0865524 ]
 [ 0.00813378  0.16787934 -0.06826946 ... -0.08552527  0.05997662
   0.16099083]]
